In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import warnings, json, astropy, os, tabulate
import astropy.io.fits as fits
from astropy.io.fits import getdata
import astropy.units as u
from astropy.coordinates import SkyCoord
warnings.filterwarnings("ignore")
from qa_gawa.qa_gawa import (
    plot_pure,
    plot_comp,
    full_completeness_distances,
    plot_clus_position,
    plots_ang_size,
    general_plots,
    recursive_print_dict,
    reading_data,
    plot_masks,
    matching_sim_det,
    write_det_numbers,
    SNR_hist,
    dist_hist,
    det_sky,
    dist_dist,
    SNR_SNR,
    puri_comp,
    plot_pure_mM,
    plot_pure_SNR,
    plot_comp_all,
    print_undet_table,
    undet_cmds
)

import tabulate
import matplotlib 
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 12}

matplotlib.rc('font', **font)

## Introduction

This jupyter notebook is intended to show plots about the ability of Gawa tto recover simulated stellar systems. <br>
The two main inputs are the list of clusters simulated and the list of detected clusters. <br>

The first task is to match the list of simulated clusters to the list of detected clusters.

Run this jupyter notebook in the LIneA env with the following command:
<br>
`jupyter nbconvert --execute --to html --EmbedImagesPreprocessor.embed_images=True qa_gawa.ipynb`
<br>
and after the command has finished, run the following cell:
<br>
 cp qa_gawa.html ~/public_html/gawa_processes/00010/detections/qa
<br>
where 00010 is the process number.

Reading the configuration file and creating a folder for the results:

In [ ]:
confg = "qa_gawa.json"

with open(confg) as fstream:
    param = json.load(fstream)

globals().update(param)

os.system("mkdir -p " + input_detection_path + "/qa")

config_gawa = input_detection_path + "/gawa.cfg"

with open(config_gawa) as fstream:
    param2 = json.load(fstream)
mask_file = param2['isochrone_masks'][param2['survey']]['model_file']

Cell below shows the list of params of the detection:

In [ ]:
recursive_print_dict(param2)

Now plotting the set of masks applied to detect simulated clusters:

In [ ]:
plot_masks(input_detection_path, mask_file, param2)

## Matching detections and simulations

### Matching with astropy search around sky function

In [ ]:
idx_sim, dix_det = matching_sim_det(input_simulation_path + '/star_clusters_simulated.dat', input_detection_path + '/clusters.fits',
                 match_file, unmatch_file, dist2match_arcmin)


## Reading the match file and showing a few plots

In [ ]:
write_det_numbers(input_detection_path, input_simulation_path, match_file, unmatch_file)

Below, a conditional is created where the clusters matched the simulated (confirmed)
and where clusters are just candidates.

In [ ]:
SNR_hist(input_detection_path, input_simulation_path, match_file, unmatch_file)

In [ ]:
dist_hist(input_detection_path, input_simulation_path, match_file, unmatch_file)

Below, a conditional is created where the clusters matched the simulated (confirmed)
and where clusters are just candidates.

In [ ]:
det_sky(input_detection_path, input_simulation_path, match_file, unmatch_file)

We can notice that all detected and true clusters (True Positives) have high signal-to-noise ratio (SNR),
while those with low SNR are false positives (FP), represented by small blue circles.

It is important to highlight in this case that all simulated clusters were detected. There may be cases where
not all simulated clusters are detected. In this case, the code should read the clusters
simulated again to see how complete the detection is.

In [ ]:
SNR_SNR(input_detection_path, input_simulation_path, match_file, unmatch_file)

In [ ]:
dist_dist(input_detection_path, input_simulation_path, match_file, unmatch_file)

## Purity of detection distance modulus

In [ ]:
puri_comp(input_detection_path, input_simulation_path, match_file, unmatch_file)

Below we will calculate the detection purity given the detected distance.

In [ ]:
plot_pure_mM(input_detection_path, input_simulation_path, match_file, unmatch_file)

In [ ]:
plot_pure_SNR(input_detection_path, input_simulation_path, match_file, unmatch_file)

## Completeness of detections

In [ ]:
plot_comp_all(input_detection_path, input_simulation_path, match_file, unmatch_file, idx_sim)

## Clusters simulated but not detected

In [ ]:
print_undet_table(unmatch_file)

In [ ]:
undet_cmds(unmatch_file, mask_file, input_simulation_path, input_detection_path, param2)

In [ ]:
general_plots(input_simulation_path + '/star_clusters_simulated.dat', unmatch_file)

In [ ]:
plot_clus_position(unmatch_file, 'ra', 'dec', input_simulation_path + '/hpx_cats_clean')

In [ ]:
plots_ang_size(input_simulation_path + '/star_clusters_simulated.dat', unmatch_file, -2.00)